## Applying Hierarchical LDA Model

Hierarchical Latent Dirichlet Allocation (hLDA) addresses the problem of learning topic hierarchies from data. The model relies on a non-parametric prior called the nested Chinese restaurant process, which allows for arbitrarily large branching factors and readily accommodates growing data collections. The hLDA model combines this prior with a likelihood that is based on a hierarchical variant of latent Dirichlet allocation [1].

Idea inspired from [Hierarchical Topic Models and the Nested Chinese Restaurant Process](http://www.cs.columbia.edu/~blei/papers/BleiGriffithsJordanTenenbaum2003.pdf)

**Reference:**

[1] David M. Blei , Michael I. Jordan , Thomas L. Griffiths , Joshua B. Tenenbaum, Hierarchical topic models and the nested chinese restaurant process, Proceedings of the 16th International Conference on Neural Information Processing Systems, p.17-24, December 09-11, 2003, Whistler, British Columbia, Canada

<font color="blue"/>

### dsp:
  * &#x1f642; Wow! That is something extra. Great!
  * To reference a scientific publication, you should give more complete data, e.g. for the 2010 version of the document: "David M. Blei, Thomas L. Griffiths, and Michael I. Jordan. 2010. The nested chinese restaurant process and bayesian nonparametric inference of topic hierarchies. J. ACM 57, 2, Article 7 (February 2010), 30 pages. DOI: https://doi.org/10.1145/1667053.1667056"

#### Hierarchical LDA Model example
![Hierarchical LDA Model example](Hierarchical_LDA_Model_example.png)

In [1]:
from gensim import corpora, models, similarities
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis.gensim

import os
import pickle
import pandas as pd
import numpy as np
import warnings

pyLDAvis.enable_notebook()

<font color="blue"/>

### dsp:
  * Sorting and adding empty lines might make this import list more accessible.

#### 1. Apply the initial LDA Model -- In order to group those documents which seem to belong to the same topic
- params:
    - processed_doc_path: cleand preprocessed data folder
    - non_german_file_path: a list of all English files
    - topic_num: topics numbers to choose while training the model
    - document_limit: restrict the number of documents to process
- variables:
    - gensim_dictionary: [Dictionay](https://radimrehurek.com/gensim/corpora/dictionary.html) type, defined by gesim library 
    - corpus: doc2bow, tokenized and nomalized list
    - lda: LDA model after training with given dataset

<font color="blue"/>

### dsp:
  * "are seemed" ~> "seem"
  * What you call a "path" here you have been calling a "folder" before.

In [2]:
def initialLDAModel(processed_doc_path, non_german_file_path, topic_num=50, document_limit=1000):
    
    #get a list of non_german files
    non_german_files = []
    with open(non_german_file_path, 'r') as fr:
        for line in fr:
            non_german_files.append(line.strip())
    
    dictionary = []
    filenames = []
    
    for root, dirs, files in os.walk(processed_doc_path):
        for f in files:
            if f[:-11] not in non_german_files:
                document_limit -= 1
                try:
                    with open(root+'/'+f, 'rb') as fr:
                        filenames.append(f)
                        document_tokens = pickle.load(fr)
                        dictionary.append(document_tokens)
                except:
                    print('Error while processing: ', f)
            
            if document_limit == 0:
                break
    
    gensim_dictionary = Dictionary(dictionary)
    corpus = [gensim_dictionary.doc2bow(text) for text in dictionary]
    lda = LdaModel(corpus, num_topics=topic_num, id2word=gensim_dictionary, iterations=200)
    topics = lda.show_topics(num_topics=-1, num_words=20)
    
    doc_pos = 0
    mat = np.zeros((len(filenames), topic_num))

    for doc in corpus:
        vector = lda[doc] # get topic probability distribution for a document
        for element in vector:
            mat[doc_pos][element[0]] = element[1]
        doc_pos += 1
        
    df = pd.DataFrame(mat, index=filenames, columns=range(0,topic_num))
    
    return df

<font color="blue"/>

### dsp:
  * The first half of this function could be a function in its own right. I think I have seen similar code in other notebooks as well.

In [3]:
%%time
warnings.filterwarnings("ignore")
#please modify the path
# non_german_file_path = '/home/bit/ma0/LabShare/data/non_german_files.txt'
# processed_doc_path = '/home/bit/ma0/LabShare/data/chui_ma/spacy_corpus/'

#relative path
non_german_file_path = './non_german_files.txt'
processed_doc_path = '../spacy_corpus/'
df = initialLDAModel(processed_doc_path, non_german_file_path, 20, 200)

CPU times: user 33.7 s, sys: 348 ms, total: 34.1 s
Wall time: 10.4 s


<font color="blue"/>

### dsp:
  * Do you know, which warning you suppress? Did you check whether it is OK to ignore them? Are there no other warnings?

#### Group topic related documents by assuming that belong to the same topic with the probability larget than 0.3
- params: 
    - df: pandas DataFrame, each column represents one topic, and each row indicates the probability of that document belongs to this topic
- variables: 
    - topic_related_documents: list, contain group of doucment names which seem to belong to the same topic

In [4]:
def get_Topic_Related_Doc(df):  
    
    topic_related_documents = []
    
    for column in range(df.shape[1]):
        row_count = 0
        for row in range(df.shape[0]):
            if df.iat[row, column] > 0.3:
                row_count += 1
        topic_related_documents.append(df.nlargest(row_count, column).index)
    
    return topic_related_documents

#### 2. Apply the HierachicalLDA LDA Model 
- params:
    - folder_path: cleand preprocessed data folder
    - non_german_file_path: a list of all English files
    - topic_num: topics numbers to choose while training the model
    - document_limit: restrict the number of documents to process
    - **topic_related_documents**: list, contain group of doucment names which seem to belong to the same topic (grouped together by high probability belonging to the same topic)
- variables:
    - gensim_dictionary: [Dictionay](https://radimrehurek.com/gensim/corpora/dictionary.html) type, defined by gesim library 
    - corpus: doc2bow, tokenized and nomalized list
    - lda: LDA model after training with given dataset

In [5]:
def hierachicalLDA(folder_path, non_german_file_path, topic_related_documents, topic_num=5, document_limit=1000):
    #get a list of non_german files
    non_german_files = []
    with open(non_german_file_path, 'r') as fr:
        for line in fr:
            non_german_files.append(line.strip())
            
    dictionary = []
    filenames = []
    for root, dirs, files in os.walk(folder_path):
        for f in sorted(files):
            if f in topic_related_documents:
                if f[:-11] not in non_german_files:
                    with open(root+'/'+f, 'rb') as fr:
                        filenames.append(f)
                        document_tokens = pickle.load(fr)
                        dictionary.append(document_tokens)

    gensim_dictionary = Dictionary(dictionary)
    corpus = [gensim_dictionary.doc2bow(text) for text in dictionary]
    lda = models.ldamodel.LdaModel(corpus, num_topics=topic_num, id2word=gensim_dictionary, iterations=200)
    topics = lda.show_topics(num_topics=-1, num_words=20)
    
    return lda, corpus, gensim_dictionary

<font color="blue"/>

### dsp:
  * This is probably the special part of this notebook.
  * You did not document the parameter `topic_related_documents`.
  * Does this parameter realized the "hierarchical" aspect of your "hierarchical LDA"?
  * You do not seem to pass a document list containing only "topic related" documents, do you?

In [6]:
docs = get_Topic_Related_Doc(df)

#### Visualize the topic model by pyLDAvis library
- params: 
    - folder_path: cleand preprocessed data folder
    - docs: group of documents after initial LDA Model
    - topic_num: topics number to choose

In [7]:
def visualization(data_path, docs, topic_num):
    
    #only visualize those topics with more than 5 documents
    if len(docs) > 5:
        lda, corpus, dictionary = hierachicalLDA(data_path, non_german_file_path, docs, topic_num)
        return pyLDAvis.gensim.prepare(lda, corpus, dictionary)
    
    else:
        print('Cannot visualize, Too few documents')

<font color="blue"/>

### dsp:
  * "Too less documents" ~> "Too few documents"

Give a simple example, apply the Hierarchical LDA on one topic that has the maximum related documents.

In [8]:
# visualization(processed_doc_path, docs[6].tolist(), 3)
doc_example = max(docs, key=len)
visualization(processed_doc_path, doc_example.tolist(), 3)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.003310  0.005276       1        1  65.363579
1     -0.004642 -0.004718       2        1  17.835274
0      0.007952 -0.000558       3        1  16.801147, topic_info=      Category         Freq            Term        Total  loglift  logprob
term                                                                      
2170   Default  2974.000000      wesentlich  2974.000000  30.0000  30.0000
211    Default  2678.000000            bank  2678.000000  29.0000  29.0000
450    Default  3125.000000         deutsch  3125.000000  28.0000  28.0000
23288  Default  3310.000000         daimler  3310.000000  27.0000  27.0000
1958   Default  2248.000000     unternehmen  2248.000000  26.0000  26.0000
2102   Default  2282.000000         vorjahr  2282.000000  25.0000  25.0000
455    Default  2687.000000        dezember  2687.000000  24.0000  24.0000
590    Default  1774.000000        ergebnis  1774.000000  23.0000  23.0000
1596   Default  2524.000000         risiken  2524.000000  22.0000  22.0000
141    Default  2014.000000        aufgrund  2014.000000  21.0000  21.0000
2032   Default  2084.000000  vermögenswerte  2084.000000  20.0000  20.0000
1761   Default  2004.000000         sonstig  2004.000000  19.0000  19.0000
943    Default  2915.000000            höhe  2915.000000  18.0000  18.0000
42407  Default   595.000000           deutz   595.000000  17.0000  17.0000
933    Default  2551.000000            hoch  2551.000000  16.0000  16.0000
588    Default  1031.000000         ergeben  1031.000000  15.0000  15.0000
575    Default  1697.000000     entwicklung  1697.000000  14.0000  14.0000
2236   Default  1391.000000        zeitwert  1391.000000  13.0000  13.0000
1815   Default  1517.000000         steuern  1517.000000  12.0000  12.0000
2670   Default  1650.000000   beizulegenden  1650.000000  11.0000  11.0000
713    Default  1136.000000     forderungen  1136.000000  10.0000  10.0000
1805   Default  1494.000000          stehen  1494.000000   9.0000   9.0000
571    Default  1583.000000     entsprechen  1583.000000   8.0000   8.0000
989    Default  1573.000000       insgesamt  1573.000000   7.0000   7.0000
988    Default  1674.000000    insbesondere  1674.000000   6.0000   6.0000
630    Default  1496.000000        erwarten  1496.000000   5.0000   5.0000
1807   Default  1103.000000         steigen  1103.000000   4.0000   4.0000
1103   Default  1560.000000        konzerns  1560.000000   3.0000   3.0000
1093   Default  2386.000000         konzern  2386.000000   2.0000   2.0000
1520   Default  1092.000000         quartal  1092.000000   1.0000   1.0000
...        ...          ...             ...          ...      ...      ...
1799    Topic3   170.244431           stark   723.050720   0.3375  -6.7432
2190    Topic3   171.807861  wirtschaftlich   736.311401   0.3284  -6.7341
1958    Topic3   420.815826     unternehmen  2248.125000   0.1081  -5.8383
741     Topic3   230.994873          führen  1072.988647   0.2479  -6.4381
3191    Topic3   226.134094      finanziell  1053.376221   0.2451  -6.4593
575     Topic3   331.106689     entwicklung  1697.711426   0.1491  -6.0780
1586    Topic3   202.854523     resultieren   922.670166   0.2689  -6.5680
2102    Topic3   408.326447         vorjahr  2282.511719   0.0628  -5.8684
588     Topic3   215.609512         ergeben  1031.567383   0.2184  -6.5070
1813    Topic3   262.163391         stellen  1338.208984   0.1536  -6.3115
1596    Topic3   428.686646         risiken  2524.586914   0.0106  -5.8198
571     Topic3   296.813416     entsprechen  1583.015869   0.1097  -6.1874
450     Topic3   500.006622         deutsch  3125.059570  -0.0489  -5.6659
2670    Topic3   300.322815   beizulegenden  1650.385010   0.0798  -6.1756
989     Topic3   289.909637       insgesamt  1573.293945   0.0924  -6.2109
1815    Topic3   281.435791         steuern  1517.527466   0.0988  -6.2406
455     Topi

<font color="blue"/>

### dsp:
  * Interesting experiment. 
  * I have still to study the literature.
  * From the code I can not understand how the "hierarchical" aspect is meant to work and I suspect you do not understand it either, or do you?